In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import Input, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import EarlyStopping, CSVLogger
from keras import metrics

import os
import numpy as np
import math

In [ ]:
num_classes = 10

In [ ]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
def get_complementary_labels(y, num_classes):
    y_c = np.zeros(len(y)).reshape(len(y), 1)
    for i,v in enumerate(y):
        y_c[i] = np.random.choice([j for j in range(num_classes) if j!=v], 1)
    return y_c

In [ ]:
y_ctrain10 = np.zeros(len(y_train)).reshape(len(y_train), 1)
y_ctrain10 = get_complementary_labels(y_train, 10)

In [ ]:
y_ctest10 = np.zeros(len(y_test)).reshape(len(y_test), 1)
y_ctest10 = get_complementary_labels(y_test, 10)

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('y_ctrain10 shape:', y_ctrain10.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('y_ctest10 shape:', y_ctest10.shape)

In [ ]:
# prepare x ordinary dataset
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Convert class vectors to binary class matrices.
y_ctrain10 = keras.utils.to_categorical(y_ctrain10, num_classes)
y_ctest10 = keras.utils.to_categorical(y_ctest10, num_classes)

print('y_train: {0}\ny_test: {1}'.format(y_train.shape, y_test.shape))
print('y_ctrain10: {0}\ny_ctest10: {1}'.format(y_ctrain10.shape, y_ctest10.shape))

In [ ]:
def vgg(x_train,classes):
    inputs = Input(shape=x_train.shape[1:])

    x = Conv2D(64, (3,3), padding="same")(inputs)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs, x)
    return model

In [ ]:
model = vgg(x_train, num_classes)

In [ ]:
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
csv_log = CSVLogger("Cifar4_normal.csv", separator=',', append=True)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-5, amsgrad=True),
              metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 100

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    zca_whitening=False,
    rotation_range=15, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False) 

datagen.fit(x_train)

In [ ]:
history = model.fit_generator(datagen.flow(x_train, y_train,
                                           batch_size=batch_size),
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              epochs=epochs,
                              validation_data=(x_test, y_test),
                              callbacks=[es_cb, csv_log])

In [ ]:
evaluation = model.evaluate(x_test, y_test)
print(evaluation)

In [ ]:
cmodel = vgg(x_train, num_classes)

In [ ]:
def sigmoid_loss(target, output):
    return 10 / (1 + math.e ** K.categorical_crossentropy(target, output))

In [ ]:
def caccuracy(target, output):
    return 1 - metrics.categorical_accuracy(target, output)

In [ ]:
batch_size = 32
epochs = 100

In [ ]:
es_cb = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
csv_log = CSVLogger("Cifar3_complementary.csv", separator=',', append=True)

cmodel.compile(loss=sigmoid_loss,
               optimizer=Adam(lr=0.00001, decay=1e-6, amsgrad=True),
               metrics=[caccuracy])

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    zca_whitening=False,
    rotation_range=15, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False) 

datagen.fit(x_train)

In [ ]:
history = cmodel.fit_generator(datagen.flow(x_train, y_ctrain10,
                                            batch_size=batch_size),
                               steps_per_epoch=x_train.shape[0] // batch_size,
                               epochs=epochs,
                               validation_data=(x_test, y_ctest10),
                               callbacks=[es_cb, csv_log])

In [ ]:
cpred = cmodel.predict(x_test)

score = 0
for i in range(len(y_test)):
    if np.argmax(y_test[i]) == np.argmax(cpred[i]):
        score += 1
print(score / len(y_test))